<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Datengrundlage" data-toc-modified-id="Datengrundlage-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Datengrundlage</a></span></li><li><span><a href="#datetime-Objekte" data-toc-modified-id="datetime-Objekte-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>datetime Objekte</a></span></li><li><span><a href="#Einlesen-der-Zeitreihe" data-toc-modified-id="Einlesen-der-Zeitreihe-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Einlesen der Zeitreihe</a></span></li><li><span><a href="#Datentyp-für-die-Datumsangaben-korrigieren" data-toc-modified-id="Datentyp-für-die-Datumsangaben-korrigieren-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Datentyp für die Datumsangaben korrigieren</a></span></li><li><span><a href="#Date-als-Index-setzen" data-toc-modified-id="Date-als-Index-setzen-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Date als Index setzen</a></span></li><li><span><a href="#Hinzufügen-weiterer-Spalten" data-toc-modified-id="Hinzufügen-weiterer-Spalten-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Hinzufügen weiterer Spalten</a></span></li><li><span><a href="#Zeitbasierte-Indizierung-mittels-Pandas" data-toc-modified-id="Zeitbasierte-Indizierung-mittels-Pandas-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Zeitbasierte Indizierung mittels Pandas</a></span></li></ul></div>

# Datengrundlage

Die folgenden Zeitreihendaten stammen von der "Open Power Systems Data" Initiative:

https://open-power-system-data.org/

Die eigentlichen Daten, mit denen wir arbeiten wollen, können hier runtergeladen werden (es sind bereits vorverarbeitete Daten):

https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

Code zum Vorverarbeitungsprojekt kann hier eingesehen werden:

https://github.com/jenfly/opsd


# datetime Objekte

Um Datums/Zeitangaben zu repräsentieren gibt es das Paket `datetime` in der Standardbibliothek. Wir schauen uns das mal kurz vorab an:

Erstmal machen wir aus einer Zeichenkette ein DateTime Objekt:

In [1]:
import pandas as pd
t = pd.to_datetime('2019-09-15 10:52pm')

In [2]:
t

Timestamp('2019-09-15 22:52:00')

Wir sehen, dass wir als Ergebnis ein Timestamp Objekt erhalten.

In [3]:
type(t)

pandas._libs.tslib.Timestamp

Wir können auch bei der Zeichenkette die Zeitangabe weglassen. Was aber macht Pandas dann daraus?

In [4]:
pd.to_datetime('1/2/1976')

Timestamp('1976-01-02 00:00:00')

Pandas geht dann davon aus, dass die 1. Zahl in der Zeichenkette der Monat ist und nicht der Tag.

Wir können Pandas aber auch mitteilen, dass wir es umgekehrt haben wollen:

In [5]:
pd.to_datetime('1/2/1976', dayfirst=True)

Timestamp('1976-02-01 00:00:00')

Jetzt ist es nicht mehr der 2. Januar, sondern der 01. Februar 1976.

# Einlesen der Zeitreihe

In [6]:
d = pd.read_csv('opsd_germany_daily.csv')

In [7]:
d.shape

(4383, 5)

In JupyterLab können wir auch über einen Doppelklick auf die .csv Datei eine Ansicht der Daten in Tabellenform erhalten. Das ist ein kleiner Vorteil von "jupyter lab" gegenüber "jupyter notebook".

Schauen wir uns mal die ersten 4 Zeilen an:

In [8]:
d.head(4)

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN


Und jetzt die letzten 4 Zeilen:

In [9]:
d.tail(4)

,Date,Consumption,Wind,Solar,Wind+Solar
4379,2017-12-28,1299.86398,506.424,14.162,520.586
4380,2017-12-29,1295.08753,584.277,29.854,614.131
4381,2017-12-30,1215.44897,721.247,7.467,728.714
4382,2017-12-31,1107.11488,721.176,19.980,741.156


Wir sehen schon: wir müssen uns darauf einstellen, dass nicht alle Zellen Daten enthalten!

In den ersten Zeilen der Tabelle scheint es noch keine Angaben zur Energieproduktion durch Windkraft- bzw. Solaranlagen zu geben.

# Datentyp für die Datumsangaben korrigieren

Was für Datentypen nimmt Pandas eigentlich, um die Einträge in den einzelnen Spalten zu speichern?

In [10]:
d.dtypes

Date            object
Consumption    float64
Wind           float64
Solar          float64
Wind+Solar     float64
dtype: object

Ups! Pandas erkennt das Datum noch nicht in der "Date" Spalte. Es repräsentiert die Datumsangaben einfach als Zeichenketten/Strings (verwirrend ist, dass das in Pandas "object" heißt).

Wir nutzen jetzt die `to_datetime()` Funktion von Pandas um in einem Rutsch die ganze Spalte "Date", die noch als Zeichenketten erkannt werden, in Datumsobjekte umzuwandeln. 

In [11]:
d["Date"] = pd.to_datetime(d["Date"])

In [12]:
d.dtypes

Date           datetime64[ns]
Consumption           float64
Wind                  float64
Solar                 float64
Wind+Solar            float64
dtype: object

Juhu! Jetzt erkennt Pandas, dass es sich in der Spalte "Date" auch um Datumsangaben handelt und "denkt" nicht, dass es einfach nur so Zeichenketten sind.

# Date als Index setzen

Bisher sind die Zeilen in der Tabelle einfach nur so durchnummeriert von 1-4383. Das ist aktuelle Index der Daten.

Aber wir wollen jetzt als Indizierung der Daten die Datumsangabe nehmen, denn wir wollen nicht z.B. später sagen:

    "Gib uns doch mal die 42. Angabe des Energieverbrauchs"
    
sondern

    "Gib uns doch mal den Energieverbrauch vom Datum=2006-02-11".

In [13]:
d = d.set_index("Date")

In [14]:
d

,Consumption,Wind,Solar,Wind+Solar
Date,,,,
2006-01-01,1069.18400,NaN,NaN,NaN
2006-01-02,1380.52100,NaN,NaN,NaN
2006-01-03,1442.53300,NaN,NaN,NaN
2006-01-04,1457.21700,NaN,NaN,NaN
2006-01-05,1477.13100,NaN,NaN,NaN
2006-01-06,1403.42700,NaN,NaN,NaN
2006-01-07,1300.28700,NaN,NaN,NaN
2006-01-08,1207.98500,NaN,NaN,NaN
2006-01-09,1529.32300,NaN,NaN,NaN


Wir können den Index uns auch explizit holen:

In [15]:
d.index

DatetimeIndex(['2006-01-01', '2006-01-02', '2006-01-03', '2006-01-04',
               '2006-01-05', '2006-01-06', '2006-01-07', '2006-01-08',
               '2006-01-09', '2006-01-10',
               ...
               '2017-12-22', '2017-12-23', '2017-12-24', '2017-12-25',
               '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29',
               '2017-12-30', '2017-12-31'],
              dtype='datetime64[ns]', name='Date', length=4383, freq=None)

# Hinzufügen weiterer Spalten

In [16]:
d["Wochentag"] = d.index.weekday_name

In [17]:
d

,Consumption,Wind,Solar,Wind+Solar,Wochentag
Date,,,,,
2006-01-01,1069.18400,NaN,NaN,NaN,Sunday
2006-01-02,1380.52100,NaN,NaN,NaN,Monday
2006-01-03,1442.53300,NaN,NaN,NaN,Tuesday
2006-01-04,1457.21700,NaN,NaN,NaN,Wednesday
2006-01-05,1477.13100,NaN,NaN,NaN,Thursday
2006-01-06,1403.42700,NaN,NaN,NaN,Friday
2006-01-07,1300.28700,NaN,NaN,NaN,Saturday
2006-01-08,1207.98500,NaN,NaN,NaN,Sunday
2006-01-09,1529.32300,NaN,NaN,NaN,Monday


In [18]:
d["Jahr"] = d.index.year
d["Monat"] = d.index.month

In [19]:
d

,Consumption,Wind,Solar,Wind+Solar,Wochentag,Jahr,Monat
Date,,,,,,,
2006-01-01,1069.18400,NaN,NaN,NaN,Sunday,2006,1
2006-01-02,1380.52100,NaN,NaN,NaN,Monday,2006,1
2006-01-03,1442.53300,NaN,NaN,NaN,Tuesday,2006,1
2006-01-04,1457.21700,NaN,NaN,NaN,Wednesday,2006,1
2006-01-05,1477.13100,NaN,NaN,NaN,Thursday,2006,1
2006-01-06,1403.42700,NaN,NaN,NaN,Friday,2006,1
2006-01-07,1300.28700,NaN,NaN,NaN,Saturday,2006,1
2006-01-08,1207.98500,NaN,NaN,NaN,Sunday,2006,1
2006-01-09,1529.32300,NaN,NaN,NaN,Monday,2006,1


Quiz: Erzeugen Sie eine Wochenendespalte!

In [37]:
mask1 = d["Wochentag"] == "Saturday"
mask2 = d["Wochentag"] == "Sunday"


d["Wochenende"] = (mask1 or mask2).any()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [29]:
d

,Consumption,Wind,Solar,Wind+Solar,Wochentag,Jahr,Monat,Wochenende
Date,,,,,,,,
2006-01-01,1069.18400,NaN,NaN,NaN,Sunday,2006,1,True
2006-01-02,1380.52100,NaN,NaN,NaN,Monday,2006,1,True
2006-01-03,1442.53300,NaN,NaN,NaN,Tuesday,2006,1,True
2006-01-04,1457.21700,NaN,NaN,NaN,Wednesday,2006,1,True
2006-01-05,1477.13100,NaN,NaN,NaN,Thursday,2006,1,True
2006-01-06,1403.42700,NaN,NaN,NaN,Friday,2006,1,True
2006-01-07,1300.28700,NaN,NaN,NaN,Saturday,2006,1,True
2006-01-08,1207.98500,NaN,NaN,NaN,Sunday,2006,1,True
2006-01-09,1529.32300,NaN,NaN,NaN,Monday,2006,1,True


# Zeitbasierte Indizierung mittels Pandas